In [2]:
import pandas as pd
import re


In [6]:
from datasets import Dataset

c:\work\ml\analyze_series_with_NLP\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Dataset

In [5]:
dataset_path = "C:/work/ml/analyze_series_with_NLP/data/GOT.csv"
df = pd.read_csv(dataset_path)

In [7]:
df = df.drop(columns=["Release Date" , "Season" , "Episode" , "Episode Title" ],axis=1)
df.head()

,Name,Sentence
0,waymar royce,What do you expect? They're savages. One lot s...
1,will,I've never seen wildlings do a thing like this...
2,waymar royce,How close did you get?
3,will,Close as any man would.
4,gared,We should head back to the wall.


In [10]:
df['Sentence'] = df['Sentence'].astype(str)

In [11]:
df['number_of_words'] = df['Sentence'].str.strip().str.split(" ")
df['number_of_words'] = df['number_of_words'].apply(lambda x: len(x))

In [12]:
df.head()

,Name,Sentence,number_of_words
0,waymar royce,What do you expect? They're savages. One lot s...,25
1,will,I've never seen wildlings do a thing like this...,21
2,waymar royce,How close did you get?,5
3,will,Close as any man would.,5
4,gared,We should head back to the wall.,7


In [14]:
df['tyrion_response_flag'] = 0
df.loc[(df['Name']=="tyrion lannister")&(df['number_of_words']>5),'tyrion_response_flag']=1

In [15]:
df

,Name,Sentence,number_of_words,tyrion_response_flag
0,waymar royce,What do you expect? They're savages. One lot s...,25,0
1,will,I've never seen wildlings do a thing like this...,21,0
2,waymar royce,How close did you get?,5,0
3,will,Close as any man would.,5,0
4,gared,We should head back to the wall.,7,0
...,...,...,...,...
23906,brienne,I think we can all agree that ships take prece...,12,0
23907,bronn,I think that's a very presumptuous statement.,7,0
23908,tyrion lannister,I once brought a jackass and a honeycomb into ...,11,1
23909,man,The Queen in the North!,5,0


In [16]:
indexes_to_take = list(df[(df['tyrion_response_flag']==1)&(df.index>0)].index)

In [17]:
indexes_to_take[:3]

[145, 147, 151]

In [18]:
system_promt = """" Your are Tyrion Lannister from the TV Show "Game of thrones". Your responses should reflect his personality and speech patterns \n"""

prompts = []
for ind in indexes_to_take:
    prompt = system_promt

    prompt += df.iloc[ind -1]['Sentence']
    prompt += '\n'
    prompt += df.iloc[ind]['Sentence']
    prompts.append(prompt)

In [19]:
print(prompts[3])

" Your are Tyrion Lannister from the TV Show "Game of thrones". Your responses should reflect his personality and speech patterns 
I hear they call him the Imp.
I hear he hates that nickname.


In [20]:
df = pd.DataFrame({"prompt":prompts})
df.head()

,prompt
0,""" Your are Tyrion Lannister from the TV Show ""..."
1,""" Your are Tyrion Lannister from the TV Show ""..."
2,""" Your are Tyrion Lannister from the TV Show ""..."
3,""" Your are Tyrion Lannister from the TV Show ""..."
4,""" Your are Tyrion Lannister from the TV Show ""..."


In [27]:
dataset = Dataset.from_pandas(df)

In [21]:
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
)

In [22]:
import huggingface_hub
from datasets import Dataset
import transformers

from peft import LoraConfig, PeftModel
from trl import SFTConfig, SFTTrainer
import gc
